# Trabalho de Conclusão de Curso

Este notebook contém a análise inicial dos conjuntos de dados para a predição de incêndios florestais no território brasileiro.

De início, foram importadas as bibliotecas necessárias para cálculos geográficos, além de ferramentas para plotagem gráfica de mapas geográficos.

In [1]:
import numpy as np
import folium
from IPython.display import display
import pandas as pd
import os

In [2]:
# Coordenadas aproximadas do centro do Brasil
latitude = -14.235
longitude = -51.925

# Criar o mapa com Folium centralizado no Brasil
mapa_brasil = folium.Map(location=[latitude, longitude], zoom_start=4)

# Exibir o mapa no notebook
display(mapa_brasil)

## Importando os datasets

Agora iremos importar os datasets, provenientes do Kaggle hub (os links estarão disponíveis no arquivo README)

In [3]:
import kagglehub

# Brazilian Cities
path = kagglehub.dataset_download("crisparada/brazilian-cities")

print("Path to dataset files:", path)

# Fire Watch Brazil 2024
path = kagglehub.dataset_download("mayaravalliero/fire-watch-brazil-2024")

print("Path to dataset files:", path)

# Geospatial Environmental and Socioeconomic Data
path = kagglehub.dataset_download("cathetorres/geospatial-environmental-and-socioeconomic-data")

print("Path to dataset files:", path)

Path to dataset files: /home/mazner/.cache/kagglehub/datasets/crisparada/brazilian-cities/versions/5
Path to dataset files: /home/mazner/.cache/kagglehub/datasets/mayaravalliero/fire-watch-brazil-2024/versions/42
Path to dataset files: /home/mazner/.cache/kagglehub/datasets/cathetorres/geospatial-environmental-and-socioeconomic-data/versions/1


## Lendo os datasets

a proxima célula define a leitura dos datasets, para podermos trabalhar com eles, ela se resume à um loop inicial que percorre os subdiretórios dentro de "datasets", e verifica se possuem arquivos .csv, caso seja verdadeiro, ele adiciona o caminho e salva em uma lista que então é utilizada no próximo loop para listar uma pré visualização dos mesmos.

In [4]:
import pandas as pd
import os

# Diretório base dos datasets
base_dir = "../datasets"

# Função para carregar todos os arquivos CSV dentro dos subdiretórios
def carregar_datasets(base_dir):
    datasets = {}
    for subdir in os.listdir(base_dir):
        subdir_path = os.path.join(base_dir, subdir)
        if os.path.isdir(subdir_path):  # Verificar se é uma pasta
            for file in os.listdir(subdir_path):
                if file.endswith(".csv"):  # Filtrar arquivos CSV
                    file_path = os.path.join(subdir_path, file)
                    try:
                        # Verificar o nome do arquivo para determinar o separador
                        if "FireWatch" in file:
                            # FireWatch usa ',' como separador
                            datasets[file] = pd.read_csv(file_path, sep=',', on_bad_lines='skip')
                        else:
                            # Brazilian Cities usa ';' como separador
                            datasets[file] = pd.read_csv(file_path, sep=';', on_bad_lines='skip')
                    except Exception as e:
                        print(f"Erro ao ler {file}: {e}")
    return datasets

# Carregar os datasets
datasets = carregar_datasets(base_dir)

# Exibir os nomes dos arquivos carregados
print("Datasets carregados:", list(datasets.keys()))

# Exemplo de visualização de um dos datasets
for nome, df in datasets.items():
    print(f"Primeiros dados de {nome}:")
    print(df.head(), "\n")


Datasets carregados: ['BRAZIL_CITIES.csv', 'Data_Dictionary.csv', 'Dataset_FireWatch_Brazil_Q1_2024.csv', 'Dataset_FireWatch_Brazil_Q2_2024.csv', 'Dataset_FireWatch_Brazil_Q3_2024.csv', 'Dataset_FireWatch_Brazil_Q4_2024.csv', 'Dataset_FireWatch_Brazil_Central_West.csv', 'central_west.csv', 'substations_attributes.csv', 'municipios_ausentes_firewatch.csv']
Primeiros dados de BRAZIL_CITIES.csv:
                  CITY STATE  CAPITAL  IBGE_RES_POP  IBGE_RES_POP_BRAS  \
0      Abadia De Goiás    GO        0        6876.0             6876.0   
1  Abadia Dos Dourados    MG        0        6704.0             6704.0   
2            Abadiânia    GO        0       15757.0            15609.0   
3               Abaeté    MG        0       22690.0            22690.0   
4           Abaetetuba    PA        0      141100.0           141040.0   

   IBGE_RES_POP_ESTR  IBGE_DU  IBGE_DU_URBAN  IBGE_DU_RURAL  IBGE_POP  ...  \
0                0.0   2137.0         1546.0          591.0    5300.0  ...   
1  

## Explorando os dados
Agora iremos explorar os dados de brazilian cities e firewatch, a fim de entender melhor os dados para podermos dar prosseguimento à tarefa.

Para a exploração dos dados, iremos realizar os passos de:
- Análise de informações;
- Análise de dados;
- Verificação de dados faltantes;
- Tratamento de dados faltantes;
 

In [5]:
# Exibir o resumo dos dados para todos os datasets
nome = "Dataset_FireWatch_Brazil_Q2_2024.csv"
print(f"Informações sobre o dataset {nome}:")
print(f"Total de amostras: {df.shape[0]}")  # Total de linhas (amostras)
print(df.info())  # Exibe informações gerais sobre o dataframe (tipos de dados, valores nulos)
print(df.describe())  # Estatísticas descritivas dos dados numéricos
print("\n")


Informações sobre o dataset Dataset_FireWatch_Brazil_Q2_2024.csv:
Total de amostras: 4260
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4260 entries, 0 to 4259
Data columns (total 1 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   municipio,media_precipitacao_10_anos,media_dias_sem_chuva_10_anos  4260 non-null   object
dtypes: object(1)
memory usage: 33.4+ KB
None
       municipio,media_precipitacao_10_anos,media_dias_sem_chuva_10_anos
count                                                4260               
unique                                               4260               
top                                      óleo,0.0,19.7225               
freq                                                    1               




In [6]:
# Verificar valores ausentes
for nome, df in datasets.items():
    print(f"Valores ausentes no dataset {nome}:")
    print(df.isnull().sum())  # Conta o número de valores ausentes por coluna
    print("\n")

# Exemplo de preenchimento de valores ausentes com a média de uma coluna numérica
# df['coluna'] = df['coluna'].fillna(df['coluna'].mean())

Valores ausentes no dataset BRAZIL_CITIES.csv:
CITY                    0
STATE                   0
CAPITAL                 0
IBGE_RES_POP            8
IBGE_RES_POP_BRAS       8
                     ... 
Wheeled_tractor        11
UBER                 5448
MAC                  5407
WAL-MART             5471
POST_OFFICES          120
Length: 81, dtype: int64


Valores ausentes no dataset Data_Dictionary.csv:
FIELD          15
DESCRIPTION    15
REFERENCE      18
UNIT           18
SOURCE         15
Unnamed: 5     95
dtype: int64


Valores ausentes no dataset Dataset_FireWatch_Brazil_Q1_2024.csv:
data                         0
municipio                    0
estado                       0
bioma                        1
avg_numero_dias_sem_chuva    0
avg_precipitacao             0
avg_risco_fogo               0
avg_frp                      0
dtype: int64


Valores ausentes no dataset Dataset_FireWatch_Brazil_Q2_2024.csv:
data                         0
municipio                    0
estado     

### Corrigindo dados faltantes
Note que o firewatch possui apenas um dado faltante em sua composição, o que torna a análise mais fácil de ser tratada, irei verificar individualmente o valor e inserir o dado empiricamente, com base em pesquisas.
Os dados faltantes encontram-se presentes no brazilian cities, que possui dados nulos. Com base em minha análise, notei que a melhor forma de tratamento destes dados seria colocando o valor 0 como atributo, já que a média poderia levar à erros de análise em locais que por exemplo:
- Não possuem centros de correio;
- Não possuem tratores rurais;
- Não possuem Wal-mart;
E assim por diante.


In [7]:
# Verificar as linhas com valores ausentes na coluna 'bioma' do dataset Q1
firewatch_q1 = datasets['Dataset_FireWatch_Brazil_Q1_2024.csv']

# Filtrar as linhas onde a coluna 'bioma' possui valores nulos
linhas_com_valores_faltantes_bioma = firewatch_q1[firewatch_q1['bioma'].isnull()]

# Exibir as linhas com valores faltantes na coluna 'bioma'
print("Linhas com valores faltantes na coluna 'bioma' do Dataset_FireWatch_Brazil_Q1_2024.csv:")
print(linhas_com_valores_faltantes_bioma)

Linhas com valores faltantes na coluna 'bioma' do Dataset_FireWatch_Brazil_Q1_2024.csv:
             data        municipio             estado bioma  \
19130  2024-01-30  LAGOA DOS PATOS  RIO GRANDE DO SUL   NaN   

       avg_numero_dias_sem_chuva  avg_precipitacao  avg_risco_fogo  avg_frp  
19130                        0.0               0.0             0.0      0.0  


In [8]:
### Verificando os biomas presentes no datasat FireWatch Q1
# Exibir os valores únicos da coluna 'bioma'
unique_bioma_values_q1 = firewatch_q1['bioma'].unique()
print(unique_bioma_values_q1)


['Amazônia' 'Cerrado' 'Caatinga' 'Mata Atlântica' 'Pampa' 'Pantanal' nan]


Corrigindo Dados Faltantes Empiricamente
O valor faltante no Firewatch se refere ao bioma de "Lagoa dos Patos", que foi preenchido com o valor "Cerrado" após pesquisa.

In [9]:
# Acessar o dataset Firewatch Q1
firewatch_q1 = datasets['Dataset_FireWatch_Brazil_Q1_2024.csv']

# Identificar a linha onde o 'bioma' está ausente para o município 'Lagoa dos Patos'
firewatch_q1.loc[(firewatch_q1['municipio'] == 'LAGOA DOS PATOS') & (firewatch_q1['bioma'].isnull()), 'bioma'] = 'Cerrado'

# Verificar se o valor foi preenchido corretamente
print(firewatch_q1[firewatch_q1['municipio'] == 'LAGOA DOS PATOS'][['municipio', 'bioma']])


             municipio    bioma
12359  LAGOA DOS PATOS  Cerrado
19130  LAGOA DOS PATOS  Cerrado
31200  LAGOA DOS PATOS  Cerrado
34041  LAGOA DOS PATOS  Cerrado


### Resolvendo Valores Faltantes no Dataset *Brazilian Cities*

Primeiramente, vamos preencher os valores ausentes com `0` em todos os datasets. Isso garante que não tenhamos problemas com dados nulos em nossa análise.


In [10]:
# Preencher os valores ausentes com 0 em todos os datasets
for name, df in datasets.items():
    df.fillna(0, inplace=True)
    print(f"Valores ausentes no dataset {name} após preenchimento com 0:")
    print(df.isnull().sum())  # Verificar se ainda há valores ausentes


Valores ausentes no dataset BRAZIL_CITIES.csv após preenchimento com 0:
CITY                 0
STATE                0
CAPITAL              0
IBGE_RES_POP         0
IBGE_RES_POP_BRAS    0
                    ..
Wheeled_tractor      0
UBER                 0
MAC                  0
WAL-MART             0
POST_OFFICES         0
Length: 81, dtype: int64
Valores ausentes no dataset Data_Dictionary.csv após preenchimento com 0:
FIELD          0
DESCRIPTION    0
REFERENCE      0
UNIT           0
SOURCE         0
Unnamed: 5     0
dtype: int64
Valores ausentes no dataset Dataset_FireWatch_Brazil_Q1_2024.csv após preenchimento com 0:
data                         0
municipio                    0
estado                       0
bioma                        0
avg_numero_dias_sem_chuva    0
avg_precipitacao             0
avg_risco_fogo               0
avg_frp                      0
dtype: int64
Valores ausentes no dataset Dataset_FireWatch_Brazil_Q2_2024.csv após preenchimento com 0:
data              

## Estatísticas Descritivas
Agora, vamos gerar as estatísticas descritivas para os dados numéricos dos datasets.

In [11]:
# Estatísticas descritivas para os dados numéricos
for name, df in datasets.items():
    if df.select_dtypes(include='number').shape[1] > 0:  # Verificar se há colunas numéricas
        print(f"Estatísticas descritivas para o dataset {name}:")
        print(df.describe())  # Exibe as estatísticas como média, desvio padrão, etc.


Estatísticas descritivas para o dataset BRAZIL_CITIES.csv:
           CAPITAL  IBGE_RES_POP  IBGE_RES_POP_BRAS  IBGE_RES_POP_ESTR  \
count  5573.000000  5.573000e+03       5.573000e+03        5573.000000   
mean      0.004845  3.422857e+04       3.415117e+04          77.393325   
std       0.069442  2.029709e+05       2.013500e+05        1794.591273   
min       0.000000  0.000000e+00       0.000000e+00           0.000000   
25%       0.000000  5.217000e+03       5.217000e+03           0.000000   
50%       0.000000  1.092700e+04       1.091600e+04           0.000000   
75%       0.000000  2.339700e+04       2.338000e+04          10.000000   
max       1.000000  1.125350e+07       1.113378e+07      119727.000000   

            IBGE_DU  IBGE_DU_URBAN  IBGE_DU_RURAL      IBGE_POP  \
count  5.573000e+03   5.573000e+03    5573.000000  5.573000e+03   
mean   1.028424e+04   8.842860e+03    1441.377355  2.755505e+04   
std    6.472003e+04   6.431366e+04    1690.857301  1.858273e+05   
min   

### Combinando os dados

Abaixo tentarei combinar os dados de tratores, pib rural no dataset firewatch Q1

### Exibindo as Colunas dos Datasets
Primeiramente, vamos exibir as colunas de ambos os datasets para identificar as informações relevantes.

In [12]:


# Exibir colunas e primeiras linhas do dataset BRAZIL_CITIES
brazil_cities = datasets['BRAZIL_CITIES.csv']
print("Colunas do dataset BRAZIL_CITIES:")
print(brazil_cities.columns)
print("\nPrimeiras linhas do dataset BRAZIL_CITIES:")
print(brazil_cities.head())

# Exibir colunas e primeiras linhas do dataset FireWatch
firewatch_q2 = datasets['Dataset_FireWatch_Brazil_Q2_2024.csv']  # Ou qualquer outro dataset FireWatch
print("\nColunas do dataset FireWatch:")
print(firewatch_q2.columns)
print("\nPrimeiras linhas do dataset FireWatch:")
print(firewatch_q2.head())

Colunas do dataset BRAZIL_CITIES:
Index(['CITY', 'STATE', 'CAPITAL', 'IBGE_RES_POP', 'IBGE_RES_POP_BRAS',
       'IBGE_RES_POP_ESTR', 'IBGE_DU', 'IBGE_DU_URBAN', 'IBGE_DU_RURAL',
       'IBGE_POP', 'IBGE_1', 'IBGE_1-4', 'IBGE_5-9', 'IBGE_10-14',
       'IBGE_15-59', 'IBGE_60+', 'IBGE_PLANTED_AREA', 'IBGE_CROP_PRODUCTION_$',
       'IDHM Ranking 2010', 'IDHM', 'IDHM_Renda', 'IDHM_Longevidade',
       'IDHM_Educacao', 'LONG', 'LAT', 'ALT', 'PAY_TV', 'FIXED_PHONES', 'AREA',
       'REGIAO_TUR', 'CATEGORIA_TUR', 'ESTIMATED_POP', 'RURAL_URBAN',
       'GVA_AGROPEC', 'GVA_INDUSTRY', 'GVA_SERVICES', 'GVA_PUBLIC',
       ' GVA_TOTAL ', 'TAXES', 'GDP', 'POP_GDP', 'GDP_CAPITA', 'GVA_MAIN',
       'MUN_EXPENDIT', 'COMP_TOT', 'COMP_A', 'COMP_B', 'COMP_C', 'COMP_D',
       'COMP_E', 'COMP_F', 'COMP_G', 'COMP_H', 'COMP_I', 'COMP_J', 'COMP_K',
       'COMP_L', 'COMP_M', 'COMP_N', 'COMP_O', 'COMP_P', 'COMP_Q', 'COMP_R',
       'COMP_S', 'COMP_T', 'COMP_U', 'HOTELS', 'BEDS', 'Pr_Agencies',
       'Pu_A

## Verificando Municípios Comuns
Agora, vamos verificar os municípios que aparecem em ambos os datasets.

In [13]:
# Verificar municípios que estão em ambos os datasets
municipios_firewatch = set(firewatch_q2['municipio'])
municipios_brazil_cities = set(brazil_cities['CITY'])

comum = municipios_firewatch.intersection(municipios_brazil_cities)
print(f"Municípios comuns entre os datasets: {len(comum)}")
print("Exemplo de municípios comuns:", list(comum)[:10])


Municípios comuns entre os datasets: 0
Exemplo de municípios comuns: []


### Normalizando o nome dos municipios
Como os nomes estão diferentes em cada dataset, sendo que no Brazilian cities está escrito normalmente e no firewatch está escrito em letras maiúsculas, é necessário normalizar os dados a fim de podermos juntar os dados (merge)

In [14]:
# Normalizar os nomes de municípios: converter todos para minúsculas
firewatch_q2['municipio'] = firewatch_q2['municipio'].str.strip().str.lower()
brazil_cities['CITY'] = brazil_cities['CITY'].str.strip().str.lower()

# Agora, tentar o merge novamente
merged_data = pd.merge(firewatch_q2, brazil_cities[['CITY', 'Wheeled_tractor', 'GVA_AGROPEC']], 
                       left_on='municipio', right_on='CITY', how='inner')

# Exibir as primeiras linhas do dataset combinado
print("Primeiras linhas do dataset combinado após normalização:")
print(merged_data.head())


Primeiras linhas do dataset combinado após normalização:
         data      municipio    estado     bioma  avg_numero_dias_sem_chuva  \
0  2024-04-01  novo aripuanã  AMAZONAS  Amazônia                       0.00   
1  2024-04-01    barreirinha  AMAZONAS  Amazônia                       0.00   
2  2024-04-01         manaus  AMAZONAS  Amazônia                       1.00   
3  2024-04-01      parintins  AMAZONAS  Amazônia                       0.00   
4  2024-04-01    itacoatiara  AMAZONAS  Amazônia                       1.25   

   avg_precipitacao  avg_risco_fogo  avg_frp           CITY  Wheeled_tractor  \
0           5007.00             0.0      6.0  novo aripuanã              0.0   
1           2849.00             0.0     57.0    barreirinha              0.0   
2           1506.00             0.0     16.0         manaus             77.0   
3           1056.00             0.0     33.0      parintins              0.0   
4           1322.25             0.0     53.0    itacoatiara         

## Criando o Mapa de Municípios Mais Ricos
Agora vamos criar um mapa com os 100 municípios mais ricos, com informações sobre tratores e risco de fogo.

In [15]:
# Remover duplicatas de municípios para garantir que apenas municípios únicos sejam considerados
top_10_municipios_rico = merged_data.drop_duplicates(subset='municipio').nlargest(10, 'GVA_AGROPEC')

# Exibir os 10 municípios mais ricos
print("10 Municípios mais ricos ruralmente:")
print(top_10_municipios_rico[['municipio', 'estado', 'GVA_AGROPEC']])


10 Municípios mais ricos ruralmente:
                  municipio              estado  GVA_AGROPEC
265                 sapezal         MATO GROSSO   1402282.11
269                 sorriso         MATO GROSSO   1361551.12
567               rio verde               GOIÁS   1294402.41
545   campo novo do parecis         MATO GROSSO   1123082.46
263              diamantino         MATO GROSSO   1107527.64
1282            campo verde         MATO GROSSO    982943.40
561                   jataí               GOIÁS    969787.08
245              nova mutum         MATO GROSSO    893162.78
2812          rio brilhante  MATO GROSSO DO SUL    857551.68
1272           nova ubiratã         MATO GROSSO    841104.83


In [16]:
from geopy.geocoders import OpenCage
import folium
import time

api_key = "604371043dad408cb87d9c467436f603"
geolocator = OpenCage(api_key, timeout=10)  # Aumentando o timeout para 10 segundos

# Ordenar o dataframe pelos 100 municípios mais ricos ruralmente
top_100_ricos = merged_data.drop_duplicates(subset='municipio').nlargest(100, 'GVA_AGROPEC')

# Criar um mapa base centrado no Brasil
mapa_brasil = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Função para adicionar marcador ao mapa
def adicionar_marcador(municipio, latitude, longitude, trator, risco_fogo):
    popup_text = f"{municipio}<br>Tratores: {trator}<br>Risco de Fogo: {risco_fogo}"
    folium.Marker([latitude, longitude], popup=popup_text, tooltip=municipio).add_to(mapa_brasil)

# Iterar sobre os dados dos 100 municípios mais ricos e adicionar marcadores ao mapa
for index, row in top_100_ricos.iterrows():
    municipio = row['municipio']
    trator = row['Wheeled_tractor']
    risco_fogo = row['avg_risco_fogo']
    
    # Obter as coordenadas de cada município usando o geolocator
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:  # Verificar se a localização foi encontrada
            latitude = location.latitude
            longitude = location.longitude
            adicionar_marcador(municipio, latitude, longitude, trator, risco_fogo)
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Exibir o mapa
mapa_brasil


In [17]:
merged_data.columns

Index(['data', 'municipio', 'estado', 'bioma', 'avg_numero_dias_sem_chuva',
       'avg_precipitacao', 'avg_risco_fogo', 'avg_frp', 'CITY',
       'Wheeled_tractor', 'GVA_AGROPEC'],
      dtype='object')

## Traçando os 100 locais com mais radiação de fogo.

Abaixo serão explicitadas as 100 localizações com mais radiação de fogo segundo o firewatch

In [18]:
api_key = "604371043dad408cb87d9c467436f603"
geolocator = OpenCage(api_key, timeout=10)  # Aumentando o timeout para 10 segundos

# Ordenar o dataframe pelos 100 municípios com maior radiação de fogo
top_100_frp = merged_data.drop_duplicates(subset='municipio').nlargest(100, 'avg_frp')

# Criar um mapa base centrado no Brasil
mapa_brasil = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Função para adicionar marcador ao mapa
def adicionar_marcador(municipio, latitude, longitude, frp, risco_fogo):
    popup_text = f"{municipio}<br>Radiação de Fogo (FRP): {frp}<br>Risco de Fogo: {risco_fogo}"
    folium.Marker([latitude, longitude], popup=popup_text, tooltip=municipio).add_to(mapa_brasil)

# Iterar sobre os dados dos 100 municípios mais impactados e adicionar marcadores ao mapa
for index, row in top_100_frp.iterrows():
    municipio = row['municipio']
    frp = row['avg_frp']
    risco_fogo = row['avg_risco_fogo']
    
    # Obter as coordenadas de cada município usando o geolocator
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:  # Verificar se a localização foi encontrada
            latitude = location.latitude
            longitude = location.longitude
            adicionar_marcador(municipio, latitude, longitude, frp, risco_fogo)
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Exibir o mapa
mapa_brasil

### Cruzando os dados

Agora, para fins de observação, iremos cruzar os dados das 100 cidades mais ricas juntamente com as 100 cidades que possuem mais radiação de fogo, serão pegas as mesmas cidades, que estão tanto no conjunto A, quanto no conjunto B.

In [19]:
# Selecionar os 100 mais ricos ruralmente
top_100_rich = merged_data.drop_duplicates(subset='municipio').nlargest(300, 'GVA_AGROPEC')

# Selecionar os 100 com maior radiação de fogo
top_100_frp = merged_data.drop_duplicates(subset='municipio').nlargest(390, 'avg_frp')

# Encontrar a interseção dos dois subconjuntos
subset_combined = top_100_rich.merge(top_100_frp, on='municipio')

# Criar um mapa base centrado no Brasil
mapa_brasil = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Função para adicionar marcador ao mapa
def adicionar_marcador(municipio, latitude, longitude, gva, frp):
    popup_text = (
        f"{municipio}<br>Valor Agropecuário: {gva}<br>Radiação de Fogo (FRP): {frp}"
    )
    folium.Marker([latitude, longitude], popup=popup_text, tooltip=municipio).add_to(mapa_brasil)

# Iterar sobre os dados do subconjunto combinado e adicionar marcadores ao mapa
for index, row in subset_combined.iterrows():
    municipio = row['municipio']
    gva = row['GVA_AGROPEC_x']  # Valor agropecuário
    frp = row['avg_frp_y']      # Radiação de fogo
    
    # Obter as coordenadas de cada município usando o geolocator
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:  # Verificar se a localização foi encontrada
            latitude = location.latitude
            longitude = location.longitude
            adicionar_marcador(municipio, latitude, longitude, gva, frp)
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Exibir o mapa
mapa_brasil

## Cruzando os dados
Agora, para fins de observação, iremos cruzar os dados das 100 cidades mais ricas juntamente com as 100 cidades que possuem mais radiação de fogo. Serão consideradas as cidades que estão tanto no conjunto A quanto no conjunto B.

In [20]:
from geopy.geocoders import OpenCage
import folium
import time

# Substitua com sua chave de API do OpenCage
api_key = "604371043dad408cb87d9c467436f603"
geolocator = OpenCage(api_key, timeout=10)

# Dicionário para converter estados por extenso para siglas
estados_para_siglas = {
    "ACRE": "AC", "ALAGOAS": "AL", "AMAPÁ": "AP", "AMAZONAS": "AM", "BAHIA": "BA", "CEARÁ": "CE",
    "DISTRITO FEDERAL": "DF", "ESPÍRITO SANTO": "ES", "GOIÁS": "GO", "MARANHÃO": "MA", 
    "MATO GROSSO": "MT", "MATO GROSSO DO SUL": "MS", "MINAS GERAIS": "MG", "PARÁ": "PA",
    "PARAÍBA": "PB", "PARANÁ": "PR", "PERNAMBUCO": "PE", "PIAUÍ": "PI", "RIO DE JANEIRO": "RJ",
    "RIO GRANDE DO NORTE": "RN", "RIO GRANDE DO SUL": "RS", "RONDÔNIA": "RO", "RORAIMA": "RR",
    "SANTA CATARINA": "SC", "SÃO PAULO": "SP", "SERGIPE": "SE", "TOCANTINS": "TO"
}

# Dicionário de estados para regiões
regioes = {
    "NORTE": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "NORDESTE": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "CENTRO-OESTE": ["GO", "MT", "MS", "DF"],
    "SUDESTE": ["ES", "MG", "RJ", "SP"],
    "SUL": ["PR", "RS", "SC"],
}

# Função para determinar a região a partir do estado
def determinar_regiao(estado):
    sigla = estados_para_siglas.get(estado.upper(), None)  # Converte para sigla
    if not sigla:
        return "DESCONHECIDO"
    for regiao, estados in regioes.items():
        if sigla in estados:
            return regiao
    return "DESCONHECIDO"

# Adicionar coluna de região ao dataframe combinado
subset_combined['regiao'] = subset_combined['estado_x'].apply(determinar_regiao)

# Criar o mapa base centrado no Brasil
mapa_brasil = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)

# Mapeamento de cores por região
cores_regiao = {
    "CENTRO-OESTE": "blue",
    "NORTE": "green",
    "NORDESTE": "orange",
    "SUDESTE": "red",
    "SUL": "purple",
    "DESCONHECIDO": "gray",
}

# Função para adicionar marcador estilizado ao mapa
def adicionar_marcador_estilizado(municipio, latitude, longitude, gva, frp, regiao):
    cor = cores_regiao.get(regiao, "black")  # Cor padrão: preto
    popup_text = (
        f"{municipio}<br>Região: {regiao}<br>Valor Agropecuário: {gva}<br>Radiação de Fogo (FRP): {frp}"
    )
    folium.Marker(
        [latitude, longitude],
        popup=popup_text,
        tooltip=municipio,
        icon=folium.Icon(color=cor),
    ).add_to(mapa_brasil)

# Iterar sobre os dados e adicionar marcadores estilizados ao mapa
for index, row in subset_combined.iterrows():
    municipio = row['municipio']
    gva = row['GVA_AGROPEC_x']  # Valor agropecuário
    frp = row['avg_frp_y']      # Radiação de fogo
    regiao = row['regiao']      # Região do município
    
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:
            latitude = location.latitude
            longitude = location.longitude
            adicionar_marcador_estilizado(municipio, latitude, longitude, gva, frp, regiao)
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Exibir o mapa
mapa_brasil


## Selecionando a região do centro-oeste

Nota-se pelo mapa acima, que a região que mais possui incêndios, é a região do centro-oeste, utilizaremos ela a partir de agora para realizar os estudos

In [21]:
from shapely.geometry import Polygon

# Filtrar os municípios do Centro-Oeste
centro_oeste = subset_combined[subset_combined['regiao'] == "CENTRO-OESTE"]

# Obter as coordenadas para cada município do Centro-Oeste
coordenadas_centro_oeste = []
for index, row in centro_oeste.iterrows():
    municipio = row['municipio']
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:
            coordenadas_centro_oeste.append((location.latitude, location.longitude))
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Criar o mapa base centrado no Centro-Oeste
mapa_centro_oeste = folium.Map(location=[-16.5, -54.5], zoom_start=5)

# Adicionar marcadores para as cidades
for lat, lon in coordenadas_centro_oeste:
    folium.Marker([lat, lon], icon=folium.Icon(color='red')).add_to(mapa_centro_oeste)

# Criar o polígono das áreas mais suscetíveis ao fogo
if coordenadas_centro_oeste:
    # Usar os pontos das coordenadas para criar um polígono
    polygon = Polygon(coordenadas_centro_oeste)
    
    # Pegar os vértices do polígono convexo (caso os pontos estejam desordenados)
    bounds = polygon.convex_hull.exterior.coords[:]
    
    # Adicionar o polígono ao mapa
    folium.Polygon(
        locations=bounds,
        color='orange',          # Cor do contorno
        fill=True,
        fill_color='yellow',     # Cor interna
        fill_opacity=0.4,        # Transparência
        popup="Área suscetível ao fogo"
    ).add_to(mapa_centro_oeste)

# Exibir o mapa
mapa_centro_oeste

#/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Central_West.csv

In [22]:
# Carregar os arquivos Firewatch
arquivos_firewatch = [
    "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Q1_2024.csv",
    "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Q2_2024.csv",
    "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Q3_2024.csv",
    "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Q4_2024.csv"
]

# Concatenar os dados
dados_firewatch = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos_firewatch], ignore_index=True)

# Obter nomes de municípios com coordenadas (assumindo que centro_oeste é o DataFrame base)
municipios_com_coordenadas = centro_oeste['municipio'].str.lower().str.strip().unique()

# Padronizar nomes no Firewatch
dados_firewatch['municipio'] = dados_firewatch['municipio'].str.lower().str.strip()

# Filtrar apenas os municípios com coordenadas dentro da área
dados_filtrados = dados_firewatch[dados_firewatch['municipio'].isin(municipios_com_coordenadas)]

# Salvar o resultado
caminho_saida = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Central_West.csv"
dados_filtrados.to_csv(caminho_saida, index=False)

print(f"Arquivo salvo com {len(dados_filtrados)} registros em:\n{caminho_saida}")


Arquivo salvo com 1798 registros em:
/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Central_West.csv


# O que foi feito até agora:

Até o momento, realizamos análise de dados em dois datasets:  
- **Firewatch**  
- **Brazil Cities**

A análise teve como objetivo redimensionar os dados para posteriormente treinar um modelo de previsão com um conjunto focado na região do **Centro-Oeste**. Essa região foi escolhida devido ao maior número de incêndios e ao maior PIB rural, ambos correlacionados com a radiação de fogo.

### Etapas realizadas até agora:

1. **Importação dos Datasets:**  
   Inicialmente, os datasets **Firewatch** e **Brazil Cities** foram importados para análise.

2. **Análise Exploratória de Dados:**  
   Foram analisados os dados de ambos os datasets para compreender melhor seus atributos e potenciais relações.

3. **Visualização de Dados:**  
   Na fase de visualização, foi realizada a filtragem dos dados para focar na subregião Centro-Oeste. Foram gerados mapas para evidenciar a radiação de fogo e o PIB rural, facilitando a análise geoespacial.

4. **Seleção de Subregião:**  
   Focamos na região Centro-Oeste, que apresentou um alto número de incêndios e alto PIB rural, além de ser uma área com bastante variabilidade climática, fundamental para os modelos de previsão.

---

## Proposta de Pesquisa

O objetivo final é **prever incêndios** com base em dados climáticos e socioeconômicos. Isso será feito utilizando aprendizado de máquina para criar modelos preditivos que possam identificar padrões em regiões propensas a incêndios, como o Centro-Oeste.


# Dataset de análise climática

Agora será feita o download do dataset de dados climáticos por hora, o dataset é composto por diversos dados climáticos, desde 1980 até 2017, será útil para traçar padrões de clima entre os locais que possuem incêndios e os climas pertencentes aos mesmos. 

In [23]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("PROPPG-PPG/hourly-weather-surface-brazil-southeast-region")

print("Path to dataset files:", path)

Path to dataset files: /home/mazner/.cache/kagglehub/datasets/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region/versions/9


## Carregando e Visualizando os Dados Climáticos

In [24]:
import pandas as pd

# Caminho para o arquivo
caminho_arquivo = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/climate/central_west.csv"

# Carregar o dataset
dados = pd.read_csv(caminho_arquivo)

# Exibir as primeiras linhas do dataset
print("Amostras iniciais do dataset:")
print(dados.head())

# Exibir total de amostras e atributos
print(f"\nTotal de amostras: {dados.shape[0]}")
print(f"Total de atributos: {dados.shape[1]}")

# Informações gerais sobre o dataset
print("\nInformações gerais:")
print(dados.info())

# Estatísticas descritivas dos dados numéricos
print("\nEstatísticas descritivas:")
print(dados.describe())


Amostras iniciais do dataset:
    index        Data   Hora  PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
0  138998  2017-12-20  14:00                               0.0   
1  138999  2017-12-20  15:00                               0.0   
2  139000  2017-12-20  16:00                               0.0   
3  139001  2017-12-20  17:00                               0.0   
4  139002  2017-12-20  18:00                               0.0   

   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  \
0                                              899.6       
1                                              899.2       
2                                              898.6       
3                                              897.7       
4                                              897.0       

   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)  \
0                                            900.0   
1                                            899.6   
2                                            899.2

In [25]:
dados.columns

Index(['index', 'Data', 'Hora', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
       'RADIACAO GLOBAL (Kj/m²)',
       'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
       'TEMPERATURA DO PONTO DE ORVALHO (°C)',
       'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
       'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)',
       'VENTO, VELOCIDADE HORARIA (m/s)', 'region', 'state', 'station',
       'station_code', 'latitude', 'longitude', 'height'],
      dtype='object')

In [26]:
dados["station_code"].value_counts().sum()


np.int64(11427120)

## Criação do Mapa de Subestações e Municípios do Centro-Oeste

In [27]:
import folium
from shapely.geometry import Point, Polygon
import pandas as pd

# Caminho para o dataset de mudanças climáticas
caminho_climate = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/climate/central_west.csv"

# Carregar o dataset de mudanças climáticas
dados_climate = pd.read_csv(caminho_climate)

# Criar um DataFrame com as subestações únicas (station, latitude, longitude)
estacoes_climate = dados_climate[['station', 'latitude', 'longitude']].drop_duplicates()

# Criar o mapa base centrado no Centro-Oeste
mapa_centro_oeste = folium.Map(location=[-16.5, -54.5], zoom_start=5)

# Filtrar os municípios do Centro-Oeste
centro_oeste = subset_combined[subset_combined['regiao'] == "CENTRO-OESTE"]

# Obter as coordenadas dos municípios do Centro-Oeste
coordenadas_centro_oeste = []
for _, row in centro_oeste.iterrows():
    municipio = row['municipio']
    try:
        location = geolocator.geocode(municipio + ", Brazil")
        if location:
            coordenadas_centro_oeste.append((location.latitude, location.longitude))
    except Exception as e:
        print(f"Erro ao geolocalizar {municipio}: {e}")

# Criar o polígono das áreas suscetíveis ao fogo
if coordenadas_centro_oeste:
    polygon = Polygon(coordenadas_centro_oeste)
    bounds = polygon.convex_hull.exterior.coords[:]
    
    # Adicionar o polígono ao mapa
    folium.Polygon(
        locations=bounds,
        color='orange',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.4,
        popup="Área suscetível ao fogo"
    ).add_to(mapa_centro_oeste)

# Selecionar subestações dentro do polígono
subestacoes_no_poligono = [] #set
for _, row in estacoes_climate.iterrows():
    lat, lon = row['latitude'], row['longitude']
    station = row['station']
    point = Point(lat, lon)
    if polygon.contains(point):
        subestacoes_no_poligono.append(row)

# Criar DataFrame com subestações dentro do polígono
df_subestacoes_poligono = pd.DataFrame(subestacoes_no_poligono)

# Adicionar as subestações ao mapa
for _, row in df_subestacoes_poligono.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']],
        popup=f"Estação: {row['station']}",
        icon=folium.Icon(color='red')
    ).add_to(mapa_centro_oeste)

# Exibir mapa
mapa_centro_oeste

# Exibir as estações selecionadas no console
print("Estações dentro do polígono das áreas suscetíveis ao fogo:")
print(df_subestacoes_poligono)


Estações dentro do polígono das áreas suscetíveis ao fogo:
                      station   latitude  longitude
9034             NOVA UBIRATA -13.686389 -54.956111
69775               QUERENCIA -12.598611 -52.194444
107702                  COXIM -18.512222 -54.735833
117292   SAO GABRIEL DO OESTE -19.420278 -54.553056
163965        GAUCHA DO NORTE -13.184722 -53.257500
...                       ...        ...        ...
2384108              ITIQUIRA -17.175051 -54.501690
2395261             SAO SIMAO -18.969142 -50.633449
2644236           CAMPO VERDE -15.531389 -55.135556
2943663          RONDONOPOLIS -16.462500 -54.580278
3061021                 IPORA -16.423056 -51.148889

[70 rows x 3 columns]


## Informações e diferenças entre firewatch e climate data

In [35]:
import pandas as pd

# --- 0. Carregar datasets ---
# Dataset climático (supondo que já está carregado como dados_climate)
# Caso não esteja:
# dados_climate = pd.read_csv("CAMINHO/DO/CLIMATE.csv")

# Dataset FireWatch filtrado para o Centro-Oeste
caminho_firewatch = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/firewatch-2024/Dataset_FireWatch_Brazil_Central_West.csv"
dados_firewatch = pd.read_csv(caminho_firewatch)

# --- 1. Contagem de amostras por estação no dataset climático ---
contagem_amostras_estacao = dados_climate['station'].value_counts().reset_index()
contagem_amostras_estacao.columns = ['station', 'total_amostras']

print("Total de amostras por estação:")
print(contagem_amostras_estacao)

# --- 2. Comparação de cidades entre datasets ---

# Normalização de nomes (todos em minúsculas e sem espaços em excesso)
cidades_climate = set(dados_climate['station'].dropna().str.lower().str.strip().unique())
cidades_firewatch = set(dados_firewatch['municipio'].dropna().str.lower().str.strip().unique())

# Diferenças
diferenca_climate_firewatch = cidades_climate - cidades_firewatch
diferenca_firewatch_climate = cidades_firewatch - cidades_climate
intersecao_cidades = cidades_climate & cidades_firewatch

# --- 3. Relatório ---
print(f"\nNúmero de cidades no dataset climático: {len(cidades_climate)}")
print(f"Número de cidades no dataset FireWatch (Centro-Oeste): {len(cidades_firewatch)}")
print(f"Número de cidades presentes em ambos: {len(intersecao_cidades)}")

print(f"\nCidades presentes em ambos os datasets ({len(intersecao_cidades)}):")
print(sorted(intersecao_cidades))

print(f"\nCidades em climático, mas não em FireWatch ({len(diferenca_climate_firewatch)}):")
print(sorted(diferenca_climate_firewatch))

print(f"\nCidades em FireWatch, mas não em climático ({len(diferenca_firewatch_climate)}):")
print(sorted(diferenca_firewatch_climate))


Arquivo salvo em: /home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/cidades_unificadas.csv
            cidade
0         agua boa
1       agua clara
2  aguas emendadas
3    alta floresta
4    alto araguaia

Número de cidades no dataset climático: 116
Número de cidades no dataset FireWatch (Centro-Oeste): 15
Número de cidades presentes em ambos: 5

Cidades presentes em ambos os datasets (5):
['ivinhema', 'maracaju', 'rio brilhante', 'sapezal', 'sorriso']

Cidades em climático, mas não em FireWatch (111):
['agua boa', 'agua clara', 'aguas emendadas', 'alta floresta', 'alto araguaia', 'alto paraiso de goias', 'alto taquari', 'amambai', 'angelica', 'apiacas', 'aquidauana', 'aragarcas', 'aral moreira', 'bandeirantes', 'bataguassu', 'bela vista', 'bonito', 'brasilandia', 'brasilia', 'brasnorte (mundo novo)', 'brasnorte (novo mundo)', 'brazlandia', 'caarapo', 'caceres', 'caiaponia', 'camapua', 'campo grande', 'campo novo dos parecis', 'campo verde', 'carlinda', 'cas

### Separando os conjuntos de dados em .csv
Abaixo foram separados os conjuntos de dados a fim de separar os arquivos da memória de execução e os colocar em memória de disco para uso posterior.

In [47]:
# Caminho base da pasta
caminho_base = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/unified"

# Exportar datasets
dados_climate.to_csv(f"{caminho_base}/dados_climate.csv", index=False, encoding="utf-8")
dados_filtrados.to_csv(f"{caminho_base}/dados_filtrados.csv", index=False, encoding="utf-8")

print("Arquivos exportados com sucesso:")
print(f"- {caminho_base}/dados_climate.csv")
print(f"- {caminho_base}/dados_filtrados.csv")


Arquivos exportados com sucesso:
- /home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/unified/dados_climate.csv
- /home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/unified/dados_filtrados.csv


In [29]:
total_amostras_firewatch = len(dados_firewatch)
print(f"Total de amostras no FireWatch (positivas): {total_amostras_firewatch}")


Total de amostras no FireWatch (positivas): 1798


### Exportando o csv

Abaixo iremos exportar todos os dados para um csv onde ficarão armazenadas as substações que serão trabalhadas

In [30]:

# Garantir que o diretório para salvar o CSV exista
caminho_diretorio = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/substations"
os.makedirs(caminho_diretorio, exist_ok=True)

# Caminho completo do arquivo CSV
caminho_csv = os.path.join(caminho_diretorio, "substations_attributes.csv")

# Exportar o DataFrame para um arquivo CSV
df_subestacoes_poligono.to_csv(caminho_csv, index=False)

print(f"Arquivo salvo em: {caminho_csv}")

Arquivo salvo em: /home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/substations/substations_attributes.csv


In [32]:
df_subestacoes_poligono

,station,latitude,longitude
9034,NOVA UBIRATA,-13.686389,-54.956111
69775,QUERENCIA,-12.598611,-52.194444
107702,COXIM,-18.512222,-54.735833
117292,SAO GABRIEL DO OESTE,-19.420278,-54.553056
163965,GAUCHA DO NORTE,-13.184722,-53.257500
...,...,...,...
2384108,ITIQUIRA,-17.175051,-54.501690
2395261,SAO SIMAO,-18.969142,-50.633449
2644236,CAMPO VERDE,-15.531389,-55.135556
2943663,RONDONOPOLIS,-16.462500,-54.580278


## Imprimindo a geolocalização das subestações
Abaixo iremos imprimir a geolocalização das subestações que estão contidas no polígono escolhido para realização da análise 
 

In [31]:
import folium
from shapely.geometry import Point, Polygon

# Criar o mapa base centrado no Centro-Oeste
mapa_centro_oeste = folium.Map(location=[-16.5, -54.5], zoom_start=5)

# Adicionar o polígono das áreas mais suscetíveis ao fogo
if not polygon.is_empty:
    bounds = polygon.convex_hull.exterior.coords[:]
    folium.Polygon(
        locations=bounds,
        color='orange',          # Cor do contorno
        fill=True,
        fill_color='yellow',     # Cor interna
        fill_opacity=0.4,        # Transparência
        popup="Área suscetível ao fogo"
    ).add_to(mapa_centro_oeste)

# Adicionar marcadores para as subestações dentro do polígono
for index, row in df_subestacoes_poligono.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station = row['station']
    folium.Marker(
        [lat, lon],
        popup=f"Estação: {station}",
        tooltip="Subestação",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mapa_centro_oeste)

# Exibir o mapa
mapa_centro_oeste


# Resumo do Processo

Neste passo do trabalho, seguimos algumas etapas principais para integrar os dados climáticos e de incêndios. O objetivo é enriquecer o dataset de incêndios com informações climáticas e socioeconômicas e, ao mesmo tempo, realizar algumas operações de análise espacial e temporal.

## 1. **Criação do Mapa Base**
O primeiro passo foi criar um mapa base com a região Centro-Oeste utilizando a biblioteca `folium`. O mapa foi centrado na região e um polígono foi desenhado para representar as áreas mais suscetíveis a incêndios.

## 2. **Adição de Subestações**
Foi realizada a seleção de subestações presentes dentro da área de risco. Para isso, verificamos as coordenadas das subestações presentes no dataset de clima e as comparamos com o polígono definido para identificar quais estão localizadas dentro da área de risco.

### Passos para isso:
- O polígono foi gerado a partir das coordenadas geográficas dos municípios da região Centro-Oeste.
- As subestações foram verificadas utilizando a geometria das coordenadas (latitude e longitude) para confirmar se estavam dentro da área de risco.

## 3. **Análise dos Dados Climáticos**
Após o mapa ser gerado, o próximo passo é realizar a análise dos dados climáticos, focando nas cidades que não aparecem no dataset de incêndios (`Firewatch`). A ideia é estimar a média de precipitação e o número médio de dias sem chuva para essas cidades nos últimos 10 anos.

### Passos para isso:
- As cidades que não aparecem no dataset `Firewatch` foram identificadas e os dados climáticos dessas cidades foram extraídos.
- Estatísticas, como a média de precipitação e a média de dias sem chuva, foram calculadas para cada cidade. A média de dias sem chuva foi calculada considerando os últimos 10 anos de dados climáticos.

## 4. **Integração com Dados do Firewatch**
O próximo passo é enriquecer o dataset de incêndios (`Firewatch`) com as informações climáticas por trimestre. Para isso, será necessário pegar as informações climáticas das subestações e calcular a média por trimestre para cada cidade presente no `Firewatch`.

## 5. **Treinamento do Modelo de SVM**
A última etapa será a construção e treinamento de um modelo SVM (Support Vector Machine) para prever a ocorrência de incêndios com base nas variáveis climáticas e socioeconômicas. Isso será feito utilizando os dados integrados.

---

### Dados negativos

Extraímos os dados das substações que não estão no firewatch, pois assumimos que não houve fogo nelas, (exemplos negativos).

### Testar só com o firewatch e com o firewatch enriquecido com os dados negativos


In [32]:
import os
import pandas as pd
from shapely.geometry import Point

# Caminho dos datasets
caminho_diretorio = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets"

caminho_firewatch = os.path.join(caminho_diretorio, "firewatch-2024")
firewatch_files = [
    "Dataset_FireWatch_Brazil_Q1_2024.csv",
    "Dataset_FireWatch_Brazil_Q2_2024.csv",
    "Dataset_FireWatch_Brazil_Q3_2024.csv",
    "Dataset_FireWatch_Brazil_Q4_2024.csv"
]

firewatch_data = pd.concat(
    [pd.read_csv(os.path.join(caminho_firewatch, file)) for file in firewatch_files],
    ignore_index=True
)
# Verificar colunas do Firewatch
print("Colunas do Firewatch:", firewatch_data.columns)

# Carregar dados das subestações
caminho_subestacoes = os.path.join(caminho_diretorio, "firewatch-2024", "substations", "substations_attributes.csv")
df_subestacoes = pd.read_csv(caminho_subestacoes)

# Verificar colunas das subestações
print("Colunas das subestações:", df_subestacoes.columns)

# Carregar dados climáticos (exemplo de arquivo, ajuste conforme necessário)
caminho_clima = os.path.join(caminho_diretorio, "climate", "central_west.csv")
dados_climate = pd.read_csv(caminho_clima)

# Renomear colunas para facilitar a manipulação
dados_climate.rename(columns={
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao",
    "station": "cidade"
}, inplace=True)

df_subestacoes.rename(columns={"station": "cidade"}, inplace=True)

# Verificar colunas dos dados climáticos
print("Colunas dos dados climáticos:", dados_climate.columns)

# Converter datas e garantir tipos corretos
if 'Data' in dados_climate.columns:
    dados_climate['Data'] = pd.to_datetime(dados_climate['Data'])

# Verificar se a coluna 'cidade' existe nos datasets antes de prosseguir
if 'cidade' in firewatch_data.columns and 'cidade' in df_subestacoes.columns:
    cidades_firewatch = set(firewatch_data['cidade'].unique())
    cidades_subestacoes = set(df_subestacoes['cidade'].unique())
    cidades_negativas = cidades_subestacoes - cidades_firewatch

    # Filtrar os dados climáticos para essas cidades
    if 'cidade' in dados_climate.columns:
        dados_negativos = dados_climate[dados_climate['cidade'].isin(cidades_negativas)]

        # Calcular estatísticas climáticas
        estatisticas_negativas = dados_negativos.groupby('cidade').agg(
            media_precipitacao=('precipitacao', 'mean'),
            dias_sem_chuva=('precipitacao', lambda x: (x == 0).sum() / 10) # Média de dias sem chuva nos últimos 10 anos
        ).reset_index()

        # Salvar os resultados
        caminho_saida = os.path.join(caminho_diretorio, "climate", "cidades_negativas.csv")
        estatisticas_negativas.to_csv(caminho_saida, index=False)

        print(f"Arquivo salvo em: {caminho_saida}")
    else:
        print("Erro: Coluna 'cidade' não encontrada nos dados climáticos.")
else:
    print("Erro: Coluna 'cidade' não encontrada nos dados do Firewatch ou subestações.")


Colunas do Firewatch: Index(['data', 'municipio', 'estado', 'bioma', 'avg_numero_dias_sem_chuva',
       'avg_precipitacao', 'avg_risco_fogo', 'avg_frp'],
      dtype='object')
Colunas das subestações: Index(['station', 'latitude', 'longitude'], dtype='object')


In [ ]:
# Filtrar os dados para cidades ausentes no FireWatch
dados_negativos = dados_climate[dados_climate['station'].str.lower().isin(cidades_negativas)]

# Estimativa da média de precipitação e dias sem chuva
dados_negativos['Data'] = pd.to_datetime(dados_negativos['Data'])

# Filtrando para os últimos 10 anos
dados_negativos_10_anos = dados_negativos[dados_negativos['Data'] > pd.to_datetime('2014-01-01')]

# Média de precipitação
media_precipitacao = dados_negativos_10_anos.groupby('station')['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].mean()

# Dias sem chuva (valor de precipitação igual a 0)
dias_sem_chuva = dados_negativos_10_anos[dados_negativos_10_anos['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] == 0].groupby('station').size()

# Mostrar resultados
print("Média de precipitação por cidade:")
print(media_precipitacao)

print("\nNúmero médio de dias sem chuva por cidade:")
print(dias_sem_chuva)


NameError: name 'cidades_negativas' is not defined

In [ ]:
# Converter a coluna 'Data' para datetime
dados_climate['Data'] = pd.to_datetime(dados_climate['Data'], errors='coerce')

# Agora, podemos filtrar os dados para o mês de Janeiro (exemplo)
dados_janeiro = dados_climate[dados_climate['Data'].dt.month == 1]

# Mostrar os dados filtrados
print("Dados de Janeiro:", dados_janeiro.head())


NameError: name 'pd' is not defined

In [ ]:
# Converter a coluna 'Data' para datetime
dados_climate['Data'] = pd.to_datetime(dados_climate['Data'])

# Agrupar por trimestre e cidade, calculando as médias
dados_climate['trimestre'] = dados_climate['Data'].dt.to_period('Q')
media_trimestral = dados_climate.groupby(['station', 'trimestre']).agg({
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'mean',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'mean',
    'RADIACAO GLOBAL (Kj/m²)': 'mean'
}).reset_index()

# Exibir as primeiras linhas da média trimestral
print(media_trimestral.head())

# Enriquecer o dataset FireWatch com as médias trimestrais
firewatch_q2 = datasets['Dataset_FireWatch_Brazil_Q2_2024.csv']  # Ou qualquer outro dataset FireWatch
firewatch_q2['municipio'] = firewatch_q2['municipio'].str.strip().str.lower()

# Juntar os dados climáticos com o FireWatch
df_enriquecido = pd.merge(firewatch_q2, media_trimestral, left_on='municipio', right_on='station', how='left')

# Exibir as primeiras linhas do dataframe enriquecido
print(df_enriquecido.head())


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
#from sklearn.impute import SimpleImputer

# Carregar o dataset
#caminho_climate = "/home/mazner/Documents/utfpr/subjects/trabalho de conclusão de curso/datasets/climate/central_west.csv"
dados_climate = pd.read_csv(caminho_climate)

# Selecionar as colunas que serão usadas como variáveis independentes (X) e dependente (y)
X = dados_climate[['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 
                  'TEMPERATURA DO PONTO DE ORVALHO (°C)', 
                  'UMIDADE RELATIVA DO AR, HORARIA (%)',
                  'VENTO, VELOCIDADE HORARIA (m/s)', 
                  'RADIACAO GLOBAL (Kj/m²)']]  # Exemplo de variáveis independentes
y = dados_climate['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']  # Exemplo de variável dependente

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imputar valores ausentes (NaN) com a média
imputer = SimpleImputer(strategy='mean')

# Aplicar o imputador nos dados
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Escalar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Treinando o modelo SVM
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)

# Previsões e avaliação do modelo
y_pred = svm_model.predict(X_test_scaled)

# Exibir as previsões
print("Previsões no conjunto de teste:", y_pred)

# Calcular o erro médio absoluto (MAE)
mae = (y_pred - y_test).abs().mean()
print(f"Erro Médio Absoluto (MAE): {mae}")
